In [11]:
import json
import requests
import pandas as pd
import xlrd
import urllib.parse

In [51]:
file = "../mockup_data/Pages/Sample clinical data_July 2022.xlsx"
file = "../mockup_data/Rayraktar/dt_Bayraktar_ClinicalMetadata_mock_2022.07.21.xlsx"
file = "../mockup_data/Subramanian/dt_Subramanian_ClinicalMetadata_mock_2022.07.12.xlsx"
file = "../mockup_data/tnbc/Integrated PABC Study Data 09222022 (2).xlsx"
writer = pd.ExcelWriter(file.replace(".xls","")+'_perols.xlsx', engine='xlsxwriter')
dfsource = pd.read_excel(file, header=1)
dfsource

,ID,Age,Race,Ethnic,Chemo,RESP,BMI,Stage,Grade,LABC,...,Vital Status,survival_2y,survival_5y,Gene Mutation,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1=CHEMO,0=NO CHEMO,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,COH,scRNAseq,NaN,na,NaN,none,DCIS/LCIS,y,54,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,COH,scRNAseq,NaN,post-recurrence,no,none,TNBC,p,49,White,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,COH,scRNAseq,NaN,na,NaN,none,DCIS/LCIS,p,57,White,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,LSU,scRNAseq,NaN,pre,NaN,none,TNBC,y,41,African American,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
for field in dfsource.columns:
    df = pd.DataFrame(columns=['id', 'ontology_name', 'iri', 'obo_id', 'label', 'description',  'type', 'is_defining_ontology' ])
    url = "https://www.ebi.ac.uk/ols/api/search?q="+urllib.parse.quote(field)+"&queryFields=label,synonym"
    responses = json.loads(requests.get(url=url).text)
    for response in responses['response']['docs']:
        if "id" in response.keys():
            id = response["id"]
        else:
            id = None
        if "ontology_name" in response.keys():
            ontology_name = response["ontology_name"]
        else:
            ontology_name = None
        if "iri" in response.keys():
            iri = response["iri"]
        else:
            iri = None
        if "obo_id" in response.keys():
            obo_id = response["obo_id"]
        else:
            obo_id = None
        if "label" in response.keys():
            label = response["label"]
        else:
            label = None
        if "description" in response.keys():
            description = "\n".join(response["description"])
        else:
            description = None
        if "type" in response.keys():
            type = response["type"]
        else:
            type = None
        if "is_defining_ontology" in response.keys():
            is_defining_ontology = response["is_defining_ontology"]
        else:
            is_defining_ontology = None
        df.loc[len(df.index)] = [id, ontology_name, iri, obo_id, label, description,  type, is_defining_ontology ]
    if len(field) >31:
        sheetname = field[:27]+"..."
    else:
        sheetname = field
    sheetname_final = sheetname.replace("/","").replace(":", "")
    df.to_excel(writer, sheet_name=sheetname_final, index=False)
    print(sheetname_final)
    worksheet = writer.sheets[sheetname_final]  # pull worksheet object
    for idx, col in enumerate(df):  # loop through all columns
        series = df[col]
        max_len = max((
            series.astype(str).map(len).max(),  # len of largest item
            len(str(series.name))  # len of column name/header
            )) + 1  # adding a little extra space
        worksheet.set_column(idx, idx, max_len)  # set column width
    #writer.save()
writer.close()

ID
Age
Race
Ethnic
Chemo
RESP
BMI
Stage
Grade
LABC
LBBC
HER2+
TNBC
Vital Status
survival_2y
survival_5y
Gene Mutation
Unnamed 17
Unnamed 18
Unnamed 19
Unnamed 20
Unnamed 21
Unnamed 22
